In [75]:
import numpy  as np
import pandas as pd
import data_tools
import matplotlib.pyplot as plt

from Models.models import *

In [48]:
ts = pd.read_csv('Preprocessed_data/ts.csv')

log_diff_prices = ts.price.values
log_diff_prices = np.array(log_diff_prices, dtype=float)
log_diff_prices = np.diff(np.log(log_diff_prices))
log_diff_prices = np.concatenate((np.array([0]), log_diff_prices))
ts['log_diff_prices'] = log_diff_prices

yearmonth = list(ts['yearmonth'])
x_label   = data_tools.from_index_to_dates(yearmonth)

ts['X'] = np.zeros(ts.shape[0])

for t in ts.index:
    if t == 0:
        continue
        
    curr_price = ts.loc[t  , 'price']
    prev_price = ts.loc[t-1, 'price'] 
    
    ts.loc[t,'X'] = residual(prev_price,curr_price,t)


In [58]:
time = ts.index[-1]

In [78]:
ts_forecast = ts.copy()

In [79]:
# Forecasting Prices M steps ahead

M = 100

for i in range(1,M+1,1):
    
    current_time = time + i
    
    prev     = ts_forecast.loc[current_time-1,  'price']
    prev_x   = ts_forecast.loc[current_time-1,  'X']
    prev_y_x = ts_forecast.loc[current_time-12, 'X']
    
    log_P      = model_log_prices(current_time,prev, prev_x, prev_y_x)
    curr_price = np.exp(log_P)
    
    ts_forecast.loc[current_time,'price'] = curr_price
    
    X = residual(prev,curr_price,current_time)
    ts_forecast.loc[current_time,'X'] = X

In [93]:
# Forecasting Storage water N steps ahead

N = 100

for i in range(1,N+1,1):
    
    current_time = time + i
    
    prev_inflow  = ts_forecast.loc[current_time - 1, 'inflow' ]
    prev_storage = ts_forecast.loc[current_time - 1, 'storage']
    
    current_storage = model_storage(model_inflow,prev_inflow, model_evaporation, current_time, prev_storage, prev_inflow)
    current_inflow  = model_inflow(current_time, prev_inflow)
    
    ts_forecast.loc[current_time,'storage'] = current_storage
    ts_forecast.loc[current_time,'inflow']  = current_inflow